# Creating consolidated metadata file for MUR SST

# step 1: create list of files

Step 2 is [here](https://github.com/cgentemann/cloud_science/blob/master/zarr_meta/cloud_mur_v41-all-step2.ipynb)

NASA JPL PODAAC has put the entire [MUR SST](https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1) dataset on AWS cloud as individual netCDF files, **but all ~7000 of them are netCDF files.**\ Accessing one file works well, but accessing multiple files is **very slow** because the metadata for each file has to be queried. Here, we create **fast access** by consolidating the metadata and accessing the entire dataset rapidly via zarr. More background on this project:
[medium article](https://medium.com/pangeo/fake-it-until-you-make-it-reading-goes-netcdf4-data-on-aws-s3-as-zarr-for-rapid-data-access-61e33f8fe685) and in this [repo](https://github.com/lsterzinger/fsspec-reference-maker-tutorial). We need help developing documentation and more test datasets. If you want to help, we are working in the [Pangeo Gitter](https://gitter.im/pangeo-data/cloud-performant-netcdf4).


To run this code:
- you need to set your AWS credentials up using `aws configure --profile esip-qhub`
- you need to set up your `.netrc` file in your home directory with your earthdata login info


Authors:
- [Chelle Gentemann](https://github.com/cgentemann)
- [Rich Signell](https://github.com/rsignell-usgs)
- [Lucas Steringzer](https://github.com/lsterzinger/)
- [Martin Durant](https://github.com/martindurant)

Credit:
- Funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program
- AWS Public Dataset [Program](https://registry.opendata.aws/mur/)
- [QuanSight](https://www.quansight.com/) for creating Qhub, [ESIP Labs ](https://www.esipfed.org/lab) for deploying it, and [AWS Sustainablity](https://aws.amazon.com/government-education/sustainability-research-credits/) for funding it!

In [ ]:
import s3fs
import requests
from urllib import request
from http.cookiejar import CookieJar
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from json import dumps
from io import StringIO
from os.path import dirname, join
import netrc
import dask.bag as db

import os
import sys
import fsspec
import ujson   # fast json
from fsspec_reference_maker.hdf import SingleHdf5ToZarr 
from fsspec_reference_maker.combine import MultiZarrToZarr
import xarray as xr
import dask
from dask.distributed import Client
import hvplot.xarray

In [ ]:
import fsspec_reference_maker
fsspec_reference_maker.__version__

- output file locations

In [ ]:
json_dir = 's3://esip-qhub/nasa/mur/jsons_all/'

## set up earthdata login credentials
- code for setting up earthdata_login from [here](https://github.com/podaac/tutorials/blob/master/notebooks/cloudwebinar/cloud_direct_access_s3.py)
- for the earthdata login to work you need to create a .netrc file on your home directory
- .netrc file contains:\
machine urs.earthdata.nasa.gov\
login 'earthdata username'\
password 'password'

In [ ]:
#########Setting up earthdata login credentials 
# this code is from https://github.com/podaac/tutorials/blob/master/notebooks/cloudwebinar/cloud_direct_access_s3.py
def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.
    Valid endpoints:
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print("There's no .netrc file or the The endpoint isn't in the netrc file")

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

###############################################################################
edl="urs.earthdata.nasa.gov"
setup_earthdata_login_auth(edl)

def begin_s3_direct_access():
    url="https://archive.podaac.earthdata.nasa.gov/s3credentials"
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})


In [ ]:
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

ebd.set_credentials(profile='esip-qhub')

profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'
ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 30
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      environment='pangeo', worker_profile='Medium Worker', 
                                      propagate_env=True)

## Create a list of all MUR files that are on the PODAAC Cloud

In [ ]:
%%time
fs = begin_s3_direct_access()
flist = []
for lyr in range(2002,2023):
    for imon in range(1,13):
        fstr = str(lyr)+str(imon).zfill(2)+'*.nc'
        files = fs.glob(join("podaac-ops-cumulus-protected/", "MUR-JPL-L4-GLOB-v4.1", fstr))
        for file in files:
            flist.append(file)
print('total number of individual netcdf files:',len(flist))

- add s3 to filenames

In [ ]:
%%time
urls = ["s3://" + f for f in flist]

so = dict(mode='rb', anon=True, default_fill_cache=False, default_cache_type='first')

- link to filesystem - BE CAREFUL HERE

In [ ]:
fs2 = fsspec.filesystem('s3', anon=False)  
#If the directory exists, remove it (and all the files)
try:
    fs2.rm(json_dir, recursive=True)
except:
    pass

In [ ]:
def gen_json(u):
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        p = u.split('/')
        date = p[4][0:8] #p[3]
        fname = p[4] #p[5]
        outf = f'{json_dir}{date}.{fname}.json'
        print(outf)
        with fs2.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

- Create all the individual files using dask

In [ ]:
%%time
_ = dask.compute(*[dask.delayed(gen_json)(u) for u in urls], retries=10);

- Create list of all the individual files

In [ ]:
flist2 = fs2.ls(json_dir)
furls = sorted(['s3://'+f for f in flist2])
print(len(furls))
furls[0]

In [ ]:
client.close(); cluster.shutdown()